# Test 3 --- Final Test for 06.11.2025

In [250]:
# Install TensorBoard for visualization
%pip install tensorboard
%pip install tabulate

# Install required packages for reinforcement learning with Atari environments
%pip install gymnasium[atari,accept-rom-license]
%pip install autorom
%pip install stable-baselines3

# Install imageio for video recording
%pip install imageio[ffmpeg]

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [251]:
# Download Atari ROMs
!AutoROM --accept-license

AutoROM will download the Atari 2600 ROMs.
They will be installed to:
	C:\Users\migue\AppData\Roaming\Python\Python313\site-packages\AutoROM\roms

Existing ROMs will be overwritten.


C:\Users\migue\AppData\Roaming\Python\Python313\site-packages\AutoROM\AutoROM.py:264: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


# Install the Gym

In [252]:
import ale_py
import gymnasium as gym

# Configure the model save folder

In [253]:
import os
save_dir = "models/"
os.makedirs(save_dir, exist_ok=True)

# Now Model

In [254]:
from dataclasses import dataclass
import argparse
import time
from datetime import datetime
import numpy as np
import collections
import typing as tt

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.tensorboard.writer import SummaryWriter

import imageio
from IPython.display import HTML, display
from base64 import b64encode

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="imageio")
os.environ["IMAGEIO_FFMPEG_LOGLEVEL"] = "quiet"  # or "quiet"

In [255]:
#dqn_model
class DQN(nn.Module):
    def __init__(self, input_shape, n_actions):
        super(DQN, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Flatten(),
        )
        size = self.conv(torch.zeros(1, *input_shape)).size()[-1]
        self.fc = nn.Sequential(
            nn.Linear(size, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )
    def forward(self, x: torch.ByteTensor):
        x = x.float() / 255.0
        return self.fc(self.conv(x))

In [256]:
#wrappers

from gymnasium import spaces
from stable_baselines3.common import atari_wrappers


class ImageToPyTorch(gym.ObservationWrapper):
    """
    ImageToPyTorch: Reorders image dimensions from (H, W, C) to (C, H, W)
    for compatibility with PyTorch convolutional layers.
    """
    def __init__(self, env):
        super(ImageToPyTorch, self).__init__(env)
        obs = self.observation_space
        assert isinstance(obs, gym.spaces.Box)
        assert len(obs.shape) == 3
        new_shape = (obs.shape[-1], obs.shape[0], obs.shape[1])
        self.observation_space = gym.spaces.Box(
            low=obs.low.min(), high=obs.high.max(),
            shape=new_shape, dtype=obs.dtype)

    def observation(self, observation):
        return np.moveaxis(observation, 2, 0)


class BufferWrapper(gym.ObservationWrapper):
    """
    BufferWrapper: Maintains a rolling window of the last `n_steps` frames
    to give the agent a sense of temporal context.
    """
    def __init__(self, env, n_steps):
        super(BufferWrapper, self).__init__(env)
        obs = env.observation_space
        assert isinstance(obs, spaces.Box)
        new_obs = gym.spaces.Box(
            obs.low.repeat(n_steps, axis=0), obs.high.repeat(n_steps, axis=0),
            dtype=obs.dtype)
        self.observation_space = new_obs
        self.buffer = collections.deque(maxlen=n_steps)

    def reset(self, *, seed: tt.Optional[int] = None, options: tt.Optional[dict[str, tt.Any]] = None):
        for _ in range(self.buffer.maxlen):
            self.buffer.append(np.zeros_like(self.env.observation_space.low))
        obs, extra = self.env.reset()
        return self.observation(obs), extra

    def observation(self, observation: np.ndarray) -> np.ndarray:
        self.buffer.append(observation)
        return np.concatenate(self.buffer)


def make_env(env_name: str, n_steps=4, render_mode=None, **kwargs):
    print(f"Creating environment {env_name}")
    env = gym.make(env_name, render_mode=render_mode, **kwargs)
    env = atari_wrappers.AtariWrapper(env, clip_reward=False, noop_max=0)
    env = ImageToPyTorch(env)
    env = BufferWrapper(env, n_steps=n_steps)
    return env

In [ ]:
# Only rerun if it hasn't been run before; Prevents resetting the experiment log

has_run = 'config_history' in globals() and 'version_counter' in globals()

# Experiment logging utility
from IPython.display import Markdown, clear_output, display
import pandas as pd
import datetime

# Persistent experiment history
if not has_run:
    config_history = []
    version_counter = 0

# Log experiment configurations
def log_experiment(config_dict, note=""):
    """Log experiment configuration and display the table."""
    global version_counter, config_history

    """Append config to table and display nicely. Only if it's new."""
    if(config_history and all(config_dict.get(k) == config_history[-1].get(k) for k in config_dict)):
        return  # Skip logging if the config is the same as the last one

    entry = {
        "version": version_counter,
        "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        **config_dict,
        "note": note
    }
    # Append the new entry to the history
    config_history.append(entry)
    version_counter += 1

    # Display the updated experiment log
    clear_output(wait=True)
    df = pd.DataFrame(config_history)
    display(Markdown(df.to_markdown(index=False)))

has_run = 'performance_history' in globals()

if not has_run:
    performance_history = []

def log_model_performance(frame_index, model_name, performance_metric_dict):
    """Log model performance with a note."""
    global performance_history

    entry = {
        "Frame Index": frame_index,
        "model_name": model_name,
        **performance_metric_dict,
        "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
    # Append the new entry to the history
    performance_history.append(entry)

    # Display the updated experiment log
    clear_output(wait=True)
    df = pd.DataFrame(performance_history)
    display(Markdown(df.to_markdown(index=False)))

def remove_last_entry():
    """Remove the last entry from the experiment log."""
    global config_history, version_counter
    if config_history:
        config_history.pop()
        version_counter -= 1
        clear_output(wait=True)
        df = pd.DataFrame(config_history)
        display(Markdown(df.to_markdown(index=False)))

def clear_table():
    """Clear the experiment log table."""
    global config_history, version_counter
    config_history = []
    version_counter = 0
    clear_output(wait=True)

def clear_performance_table():
    """Clear the performance log table."""
    global performance_history
    performance_history = []
    clear_output(wait=True)

def print_table_for_github():
    """Print the experiment log table in markdown format for GitHub."""
    df = pd.DataFrame(config_history)
    print(df.to_markdown(index=False))

In [258]:
# Removes the last entry entered; only if I mean to
can_remove = False

if can_remove:
    remove_last_entry()

In [ ]:
# Clears the whole table, only if I mean to
can_clear = False

if can_clear:
    clear_table()

In [273]:
# Prints the table in markdown format
print_table_for_github()

In [261]:
# Base Configuration
config = {
    "DEFAULT_ENV_NAME": "ALE/Pong-v5",
    "MEAN_REWARD_BOUND": 19,

    "GAMMA": 0.99,
    "BATCH_SIZE": 32,
    "REPLAY_SIZE": 10000,
    "LEARNING_RATE": 1e-4,
    "SYNC_TARGET_FRAMES": 1000,
    "REPLAY_START_SIZE": 10000,

    "SAVE_EPSILON": 0.5,  # Only save if at least this much better
    "EPSILON_DECAY_LAST_FRAME": 150000,
    "EPSILON_START": 1.0,
    "EPSILON_FINAL": 0.01
}

# Log automatically when you rerun this cell
log_experiment(config, note="Baseline run")

# Tuple of tensors returned from a sampled minibatch in replay buffer
State = np.ndarray
Action = int
BatchTensors = tt.Tuple[
    torch.ByteTensor,           # current state
    torch.LongTensor,           # actions
    torch.Tensor,               # rewards
    torch.BoolTensor,           # done || trunc
    torch.ByteTensor            # next state
]

|   version | timestamp           | DEFAULT_ENV_NAME   |   MEAN_REWARD_BOUND |   GAMMA |   BATCH_SIZE |   REPLAY_SIZE |   LEARNING_RATE |   SYNC_TARGET_FRAMES |   REPLAY_START_SIZE |   SAVE_EPSILON |   EPSILON_DECAY_LAST_FRAME |   EPSILON_START |   EPSILON_FINAL | note         |
|----------:|:--------------------|:-------------------|--------------------:|--------:|-------------:|--------------:|----------------:|---------------------:|--------------------:|---------------:|---------------------------:|----------------:|----------------:|:-------------|
|         0 | 2025-11-12 08:17:29 | ALE/Pong-v5        |                  19 |    0.99 |           32 |         10000 |          0.0001 |                 1000 |               10000 |            0.5 |                     150000 |               1 |            0.01 | Baseline run |
|         1 | 2025-11-12 08:23:33 | ALE/Pong-v5        |                   5 |    0.99 |           32 |         10000 |          0.0001 |                  500 |                1000 |            0.5 |                      10000 |               1 |            0.01 | Quick run    |
|         2 | 2025-11-12 08:23:40 | ALE/Pong-v5        |                  19 |    0.99 |           32 |         10000 |          0.0001 |                 1000 |               10000 |            0.5 |                     150000 |               1 |            0.01 | Baseline run |
|         3 | 2025-11-12 08:50:55 | ALE/Pong-v5        |                   5 |    0.99 |           32 |         10000 |          0.0001 |                  500 |                1000 |            0.5 |                      10000 |               1 |            0.01 | Quick run    |
|         4 | 2025-11-12 10:25:28 | ALE/Pong-v5        |                  19 |    0.99 |           32 |         10000 |          0.0001 |                 1000 |               10000 |            0.5 |                     150000 |               1 |            0.01 | Baseline run |
|         5 | 2025-11-12 10:25:28 | ALE/Pong-v5        |                   5 |    0.99 |           32 |         10000 |          0.0001 |                  500 |                1000 |            0.5 |                      10000 |               1 |            0.01 | Quick run    |
|         6 | 2025-11-12 10:26:12 | ALE/Pong-v5        |                  19 |    0.99 |           32 |         10000 |          0.0001 |                 1000 |               10000 |            0.5 |                     150000 |               1 |            0.01 | Baseline run |
|         7 | 2025-11-12 10:26:12 | ALE/Pong-v5        |                   5 |    0.99 |           32 |         10000 |          0.0001 |                  500 |                1000 |            0.5 |                      10000 |               1 |            0.01 | Quick run    |
|         8 | 2025-11-12 11:02:14 | ALE/Pong-v5        |                  19 |    0.99 |           32 |         10000 |          0.0001 |                 1000 |               10000 |            0.5 |                     150000 |               1 |            0.01 | Baseline run |

In [262]:
# ⚙️ Fast Training Config for Quick Test Run
config["MEAN_REWARD_BOUND"] = 5
config["REPLAY_START_SIZE"] = 1000
config["EPSILON_DECAY_LAST_FRAME"] = 10000
config["SYNC_TARGET_FRAMES"] = 500

# config["REPLAY_SIZE"] = 5000  # optional
# config["BATCH_SIZE"] = 16     # optional

# Log automatically when you rerun this cell
log_experiment(config, note="Quick run")

|   version | timestamp           | DEFAULT_ENV_NAME   |   MEAN_REWARD_BOUND |   GAMMA |   BATCH_SIZE |   REPLAY_SIZE |   LEARNING_RATE |   SYNC_TARGET_FRAMES |   REPLAY_START_SIZE |   SAVE_EPSILON |   EPSILON_DECAY_LAST_FRAME |   EPSILON_START |   EPSILON_FINAL | note         |
|----------:|:--------------------|:-------------------|--------------------:|--------:|-------------:|--------------:|----------------:|---------------------:|--------------------:|---------------:|---------------------------:|----------------:|----------------:|:-------------|
|         0 | 2025-11-12 08:17:29 | ALE/Pong-v5        |                  19 |    0.99 |           32 |         10000 |          0.0001 |                 1000 |               10000 |            0.5 |                     150000 |               1 |            0.01 | Baseline run |
|         1 | 2025-11-12 08:23:33 | ALE/Pong-v5        |                   5 |    0.99 |           32 |         10000 |          0.0001 |                  500 |                1000 |            0.5 |                      10000 |               1 |            0.01 | Quick run    |
|         2 | 2025-11-12 08:23:40 | ALE/Pong-v5        |                  19 |    0.99 |           32 |         10000 |          0.0001 |                 1000 |               10000 |            0.5 |                     150000 |               1 |            0.01 | Baseline run |
|         3 | 2025-11-12 08:50:55 | ALE/Pong-v5        |                   5 |    0.99 |           32 |         10000 |          0.0001 |                  500 |                1000 |            0.5 |                      10000 |               1 |            0.01 | Quick run    |
|         4 | 2025-11-12 10:25:28 | ALE/Pong-v5        |                  19 |    0.99 |           32 |         10000 |          0.0001 |                 1000 |               10000 |            0.5 |                     150000 |               1 |            0.01 | Baseline run |
|         5 | 2025-11-12 10:25:28 | ALE/Pong-v5        |                   5 |    0.99 |           32 |         10000 |          0.0001 |                  500 |                1000 |            0.5 |                      10000 |               1 |            0.01 | Quick run    |
|         6 | 2025-11-12 10:26:12 | ALE/Pong-v5        |                  19 |    0.99 |           32 |         10000 |          0.0001 |                 1000 |               10000 |            0.5 |                     150000 |               1 |            0.01 | Baseline run |
|         7 | 2025-11-12 10:26:12 | ALE/Pong-v5        |                   5 |    0.99 |           32 |         10000 |          0.0001 |                  500 |                1000 |            0.5 |                      10000 |               1 |            0.01 | Quick run    |
|         8 | 2025-11-12 11:02:14 | ALE/Pong-v5        |                  19 |    0.99 |           32 |         10000 |          0.0001 |                 1000 |               10000 |            0.5 |                     150000 |               1 |            0.01 | Baseline run |
|         9 | 2025-11-12 11:02:15 | ALE/Pong-v5        |                   5 |    0.99 |           32 |         10000 |          0.0001 |                  500 |                1000 |            0.5 |                      10000 |               1 |            0.01 | Quick run    |

In [263]:
import os
from pathlib import Path

# Define directories
save_dir_local = "saved_models"

# Create both directories if they don't exist
os.makedirs(save_dir_local, exist_ok=True)

# Safe model filename
env_name = config["DEFAULT_ENV_NAME"]
safe_env_name = env_name.replace("/", "_")

In [264]:
@dataclass
class Experience:
    state: State
    action: Action
    reward: float
    done_trunc: bool
    new_state: State


class ExperienceBuffer:
    def __init__(self, capacity: int):
        self.buffer = collections.deque(maxlen=capacity)

    def __len__(self):
        return len(self.buffer)

    def append(self, experience: Experience):
        self.buffer.append(experience)

    def sample(self, batch_size: int) -> tt.List[Experience]:
        indices = np.random.choice(len(self), batch_size, replace=False)
        return [self.buffer[idx] for idx in indices]

In [265]:
class Agent:
    def __init__(self, env: gym.Env, exp_buffer: ExperienceBuffer):
        self.env = env
        self.exp_buffer = exp_buffer
        self.state: tt.Optional[np.ndarray] = None
        self._reset()

    def _reset(self):
        self.state, _ = self.env.reset()
        self.total_reward = 0.0

    @torch.no_grad()
    def play_step(self, net: DQN, device: torch.device,
                  epsilon: float = 0.0) -> tt.Optional[float]:
        done_reward = None

        if np.random.random() < epsilon:
            action = env.action_space.sample()
        else:
            state_v = torch.as_tensor(self.state).to(device)
            state_v.unsqueeze_(0)
            q_vals_v = net(state_v)
            _, act_v = torch.max(q_vals_v, dim=1)
            action = int(act_v.item())

        # do step in the environment
        new_state, reward, is_done, is_tr, _ = self.env.step(action)
        self.total_reward += reward

        exp = Experience(
            state=self.state, action=action, reward=float(reward),
            done_trunc=is_done or is_tr, new_state=new_state
        )
        self.exp_buffer.append(exp)
        self.state = new_state
        if is_done or is_tr:
            done_reward = self.total_reward
            self._reset()
        return done_reward

In [266]:
def batch_to_tensors(batch: tt.List[Experience], device: torch.device) -> BatchTensors:
    states, actions, rewards, dones, new_state = [], [], [], [], []
    for e in batch:
        states.append(e.state)
        actions.append(e.action)
        rewards.append(e.reward)
        dones.append(e.done_trunc)
        new_state.append(e.new_state)
    states_t = torch.as_tensor(np.asarray(states))
    actions_t = torch.LongTensor(actions)
    rewards_t = torch.FloatTensor(rewards)
    dones_t = torch.BoolTensor(dones)
    new_states_t = torch.as_tensor(np.asarray(new_state))
    return states_t.to(device), actions_t.to(device), rewards_t.to(device), \
           dones_t.to(device),  new_states_t.to(device)

In [267]:
def calc_loss(batch: tt.List[Experience], net: DQN, tgt_net: DQN,
              device: torch.device) -> torch.Tensor:
    states_t, actions_t, rewards_t, dones_t, new_states_t = batch_to_tensors(batch, device)

    state_action_values = net(states_t).gather(
        1, actions_t.unsqueeze(-1)
    ).squeeze(-1)

    # Decide whether to use Double DQN
    USE_DOUBLE_DQN = False

    # Compute the expected Q values
    with torch.no_grad():
        if USE_DOUBLE_DQN: # Double DQN
            next_actions = net(new_states_t).argmax(dim=1)
            next_state_values = tgt_net(new_states_t).gather(1, next_actions.unsqueeze(-1)).squeeze(-1)
        else: # DQN
            next_state_values = tgt_net(new_states_t).max(1)[0]

        next_state_values[dones_t] = 0.0

    expected_state_action_values = rewards_t + config["GAMMA"] * next_state_values; 

    return nn.MSELoss()(state_action_values, expected_state_action_values)

In [268]:
def record_policy_video(env_name, model, device, video_filename="policy_rollout.mp4", max_steps=2000):
    """Run one greedy episode using the given model and save a video."""
    # Use your make_env to preserve preprocessing
    video_env = make_env(env_name, render_mode="rgb_array")
    state, _ = video_env.reset(seed=42)
    frames, total_reward = [], 0.0

    for step in range(max_steps):
        # Capture a rendered RGB frame (already handled by render_mode)
        frame = video_env.render()
        if isinstance(frame, np.ndarray):
            frames.append(frame)

        # Greedy action
        state_v = torch.as_tensor(state).unsqueeze(0).to(device)
        q_vals_v = model(state_v)
        _, act_v = torch.max(q_vals_v, dim=1)
        action = int(act_v.item())

        state, reward, terminated, truncated, _ = video_env.step(action)
        total_reward += reward
        if terminated or truncated:
            frames.append(video_env.render())
            break

    video_env.close()

    # Save video
    if frames:
        os.makedirs(os.path.dirname(video_filename), exist_ok=True)

        # 🔹 Pad frames so height is divisible by 16 (prevents FFMPEG warnings)
        padded_frames = []
        for f in frames:
            h, w = f.shape[:2]
            new_h = (h + 15) // 16 * 16  # round up to nearest multiple of 16
            if new_h != h:
                pad = np.zeros((new_h - h, w, 3), dtype=f.dtype)
                f = np.vstack([f, pad])
            padded_frames.append(f)

        # 🔹 Write padded frames quietly
        with imageio.get_writer(video_filename, fps=30) as writer:
            for f in padded_frames:
                writer.append_data(f)

        print(f"🎬 Saved gameplay video: {video_filename} | Total reward: {total_reward:.2f}")
    else:
        print("⚠️ No frames captured — render_mode may not be supported.")



In [271]:
# Training setup
model_comment = f"test_epsdec{config["EPSILON_DECAY_LAST_FRAME"]}_rs{config["REPLAY_START_SIZE"]}_sync{config["SYNC_TARGET_FRAMES"]}"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Environment and networks
env = make_env(env_name)
net = DQN(env.observation_space.shape, env.action_space.n).to(device)
tgt_net = DQN(env.observation_space.shape, env.action_space.n).to(device)
writer = SummaryWriter(comment=f"-{env_name}-{model_comment}")
print(net)

# Experience replay buffer and agent
buffer = ExperienceBuffer(config["REPLAY_SIZE"])
agent = Agent(env, buffer)
epsilon = config["EPSILON_START"]

# Optimizer setup
optimizer = optim.Adam(net.parameters(), lr=config["LEARNING_RATE"])
total_rewards = []
frame_idx = 0
ts_frame = 0
ts = time.time()
best_m_reward = None

USE_DOUBLE_DQN = False  # Set to True to use Double DQN

if USE_DOUBLE_DQN:
    model_name = "Double DQN"
else:
    model_name = "DQN"

performance_metric = {
    "Finished Games": 0,
    "Reward": 0.0,
    "Epsilon": 0.0,
    "Speed (f/s)": 0.0,
    "Elapsed Time (min)": 0.0
}

# Training loop
start_time = time.time()
while True:
    frame_idx += 1
    epsilon = max(config["EPSILON_FINAL"], config["EPSILON_START"] - frame_idx / config["EPSILON_DECAY_LAST_FRAME"])

    reward = agent.play_step(net, device, epsilon)
    if reward is not None:
        total_rewards.append(reward)
        speed = (frame_idx - ts_frame) / (time.time() - ts)
        elapsed = time.time() - start_time  # in seconds
        ts_frame = frame_idx
        ts = time.time()
        m_reward = np.mean(total_rewards[-100:])
        #  print(f"{frame_idx}: done {len(total_rewards)} games, reward {m_reward:.3f}, "
        #      f"eps {epsilon:.2f}, speed {speed:.2f} f/s, time {elapsed/60:.1f} min")
        writer.add_scalar("epsilon", epsilon, frame_idx)
        writer.add_scalar("speed", speed, frame_idx)
        writer.add_scalar("reward_100", m_reward, frame_idx)
        writer.add_scalar("reward", reward, frame_idx)
        if best_m_reward is None or m_reward > best_m_reward + config["SAVE_EPSILON"]:
            print(f"{frame_idx}: done {len(total_rewards)} games, reward {m_reward:.3f}, "
                f"eps {epsilon:.2f}, speed {speed:.2f} f/s, time {elapsed/60:.1f} min")
            
            # Update performance metrics
            performance_metric["Finished Games"] = len(total_rewards)
            performance_metric["Reward"] = m_reward
            performance_metric["Epsilon"] = epsilon
            performance_metric["Speed (f/s)"] = speed
            performance_metric["Elapsed Time (min)"] = elapsed / 60.0

            # Log model performance
            log_model_performance(frame_idx, model_name, performance_metric)

            # Save the model
            timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M")
            model_filename = f"{safe_env_name}-best_{int(m_reward)}-{timestamp}-{model_comment}.dat"
            model_path_local = os.path.join(save_dir_local, model_filename)

            torch.save(net.state_dict(), model_path_local)

            print(f"Model saved to:\n - Local:        {model_path_local}")
            if best_m_reward is not None:
                print(f"Best reward updated {best_m_reward:.3f} -> {m_reward:.3f}")
            best_m_reward = m_reward

            # 🎥 Automatically record a short greedy rollout video
            video_filename = f"videos/{safe_env_name}-{int(m_reward)}-{timestamp}.mp4"
            os.makedirs("videos", exist_ok=True)
            record_policy_video(env_name, net, device, video_filename)
        if m_reward > config["MEAN_REWARD_BOUND"]:
            print("Solved in %d frames!" % frame_idx)
            break
    if len(buffer) < config["REPLAY_START_SIZE"]:
        continue
    if frame_idx % config["SYNC_TARGET_FRAMES"] == 0:
        tgt_net.load_state_dict(net.state_dict())

    optimizer.zero_grad()
    batch = buffer.sample(config["BATCH_SIZE"])
    loss_t = calc_loss(batch, net, tgt_net, device)
    loss_t.backward()
    optimizer.step()
env.close()
writer.close()

|   Frame Index | model_name   |   Finished Games |   Reward |   Epsilon |   Speed (f/s) |   Elapsed Time (min) | timestamp           |
|--------------:|:-------------|-----------------:|---------:|----------:|--------------:|---------------------:|:--------------------|
|           218 | DQN          |                1 |   -20    |    0.9782 |      269.29   |            0.0134627 | 2025-11-12 11:05:17 |
|         56135 | DQN          |              181 |   -19.47 |    0.01   |       32.3526 |           29.5246    | 2025-11-12 11:34:48 |
|         62737 | DQN          |              193 |   -18.94 |    0.01   |       31.6536 |           33.0653    | 2025-11-12 11:38:20 |
|         69015 | DQN          |              204 |   -18.43 |    0.01   |       30.8131 |           36.437     | 2025-11-12 11:41:42 |
|         76055 | DQN          |              216 |   -17.9  |    0.01   |       31.0258 |           40.3123    | 2025-11-12 11:45:35 |
|         82799 | DQN          |              227 |   -17.38 |    0.01   |       30.2709 |           44.1065    | 2025-11-12 11:49:23 |

Model saved to:
 - Local:        saved_models\ALE_Pong-v5-best_-17-20251112-1149-test_epsdec10000_rs1000_sync500.dat
Best reward updated -17.900 -> -17.380
Creating environment ALE/Pong-v5
🎬 Saved gameplay video: videos/ALE_Pong-v5--17-20251112-1149.mp4 | Total reward: -10.00


KeyboardInterrupt: 